<a href="https://colab.research.google.com/github/SelinErcan/DataMining/blob/main/n_gram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Libraries

Source code for seperating a given Turkish word into syllable: https://github.com/ftkurt/python-syllable

In [ ]:
!pip install git+https://github.com/ftkurt/python-syllable.git@master

  Cloning https://github.com/ftkurt/python-syllable.git (to revision master) to /tmp/pip-req-build-_ya7o1vo
  Running command git clone -q https://github.com/ftkurt/python-syllable.git /tmp/pip-req-build-_ya7o1vo
     |████████████████████████████████| 286kB 3.5MB/s 
     |████████████████████████████████| 23.2MB 99kB/s 
     |████████████████████████████████| 245kB 48.2MB/s 
     |████████████████████████████████| 737kB 40.8MB/s 
  Created wheel for syllable: filename=syllable-0.0.1-cp36-none-any.whl size=209758 sha256=fa5d813d8ea8a5b1486cb184018e3af38b4285cd5b18a16764172ced9218fcc2
  Stored in directory: /tmp/pip-ephem-wheel-cache-66o7poic/wheels/a0/c2/74/545cd8f99f5bea7228d247e69f8b536e40c6c145d9cf60312d
Successfully built syllable
ERROR: chainer 6.5.0 has requirement typing-extensions<=3.6.6, but you'll have typing-extensions 3.7.4.1 which is incompatible.
  Found existing installation: typing-extensions 3.6.6
    Uninstalling typing-extensions-3.6.6:
      Successfully uninstalled

In [ ]:
import nltk
nltk.download('punkt')
from nltk import ngrams
from syllable import Encoder
from google.colab import files
import io
import re
import pandas as pd
import numpy as np
from nltk import FreqDist, MLEProbDist
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.tokenize import word_tokenize, WhitespaceTokenizer, TweetTokenizer
np.random.seed(seed=234)
from math import log
from collections import OrderedDict
from operator import itemgetter 
import random
from math import log

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Read trwiki file and split the data

In [ ]:
# The below reads in N lines of text and divides it into training and test text.
N = 100
from google.colab import drive
drive.mount('/content/drive')
file = 'drive/My Drive/trwiki.txt'
with open(file) as myfile:
    all_articles = [next(myfile) for x in range(N)]
#print(all_articles)
articles = all_articles[0:95]
test_articles = all_articles[95:N]
joined_articles = [" ".join(articles)]
joined_test_articles = [" ".join(test_articles)]# The below takes out apostrophes (don't becomes dont), replacing anything that's not a letter with a space. Any single letter that is not the pronoun "I" or the article "a" is also replaced with a space, even at the beginning or end of a document.
print(joined_articles)
print(joined_test_articles)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
['<doc id="10" url="https://tr.wikipedia.org/wiki?curid=10" title="Cengiz Han">\n Cengiz Han\n \n Cengiz Han ("Cenghis Khan", "Çinggis Haan" ya da doğum adıyla Temuçin (anlamı: demirci), Moğolca: "Чингис Хаан" ya da "Tengiz" (anlamı: deniz), ; d. 1162 – ö. 18 Ağustos 1227), Moğol komutan, hükümdar ve Moğol İmparatorluğu\'nun kurucusudur. Cengiz Han, 13. Yüzyılın başında Orta Asya\'daki tüm göçebe bozkır kavimlerini birleştirerek bir ulus haline getirdi ve o ul

## Use syllables library 

In [ ]:
encoder = Encoder(lang="tr", limitby="vocabulary", limit=3000)  # split into syllables (also cleans the text)

tokens= joined_articles[0].split()
print("split words:")
print(tokens)

train_syllables=""
for words in tokens:
  if encoder.tokenize(words) != "":
    train_syllables += encoder.tokenize(words).replace(" ", ",") + ", ,"
    #print(train_syllables)
train_syllables = train_syllables[:-3]
print("\n split syllables:")
print(train_syllables)

split words:
['<doc', 'id="10"', 'url="https://tr.wikipedia.org/wiki?curid=10"', 'title="Cengiz', 'Han">', 'Cengiz', 'Han', 'Cengiz', 'Han', '("Cenghis', 'Khan",', '"Çinggis', 'Haan"', 'ya', 'da', 'doğum', 'adıyla', 'Temuçin', '(anlamı:', 'demirci),', 'Moğolca:', '"Чингис', 'Хаан"', 'ya', 'da', '"Tengiz"', '(anlamı:', 'deniz),', ';', 'd.', '1162', '–', 'ö.', '18', 'Ağustos', '1227),', 'Moğol', 'komutan,', 'hükümdar', 've', 'Moğol', "İmparatorluğu'nun", 'kurucusudur.', 'Cengiz', 'Han,', '13.', 'Yüzyılın', 'başında', 'Orta', "Asya'daki", 'tüm', 'göçebe', 'bozkır', 'kavimlerini', 'birleştirerek', 'bir', 'ulus', 'haline', 'getirdi', 've', 'o', 'ulusu', '"Moğol"', 'siyasi', 'kimliği', 'çatısı', 'altında', 'topladı.', 'Dünya', 'tarihinin', 'en', 'büyük', 'askeri', 'dehalarından', 'biri', 'olarak', 'kabul', 'edilen', 'Cengiz', 'Han,', 'hükümdarlığı', 'döneminde', '1206-1227', 'arasında', 'Kuzey', "Çin'deki", 'Batı', 'Xia', 've', 'Jin', 'Hanedanı,', "Türkistan'daki", 'Kara', 'Hıtay,', 'Maveraü

## Find N-Grams

In [ ]:
def extract_ngrams(data,n):
  n_grams = ngrams(data.split(','),n)
  fdist = nltk.FreqDist(n_grams)
  prob_dist = nltk.MLEProbDist(fdist)
  ngram_list = {}
  for k,v in fdist.items():
    ngram_list[k] = v
  return ngram_list

## Funtion to calculate good_turing counts

In [ ]:
MINUS_INFINITY_SENTENCE_LOG_PROB = -1000
def GoodTuring(grams, totalNumberOfGrams,fileName):
    listOfProb = {}
    bucket = {}
    bucketList = []
    cStar = {}
    pStar = {}
    listOfCounts = {}

    i = 1
    bucket[0] = 0;
    for key,value in grams.items():
        if not value in bucket:
            bucket[value] = 1
        else:
            bucket[value] += 1
    bucket[0] = bucket[1]

    # Sorted Bucket
    bucketList = sorted(bucket.items() , key=lambda t : t[0])
    bucketList.append(bucketList[len(bucketList)-1])
    zeroOccurenceProb = bucketList[0][1] / totalNumberOfGrams
    lastItem = bucketList[len(bucketList)-1][0]
    lenBucketList = len(bucketList)

    #print(lenBucketList)
    for k, v in bucketList:
        if i <= lenBucketList-1:
            if v == 0:
                cStar[k] = 0
                pStar[k] = 0

            else:
                cStar[k] = (i+1) * bucketList[i][1] / v              
                pStar[k] = cStar[k] / totalNumberOfGrams

        i += 1

    #print(gram[0])
    #print(cStar.get(grams[gram[0]]))
    for key, value in grams.items():
        listOfProb[key] = pStar.get(grams[key])
        listOfCounts[key] = cStar.get(grams[key])

    file = open(fileName, 'w',encoding="utf8")
    file.write('Gram' + '\t\t\t' + 'Count' + '\t' + 'Probability' + '\n')

    for key, value in grams.items():
        file.write(str(key) + ' : ' + str(grams[key])
                    + ' : ' + str(listOfProb[key]) + '\n')
    file.close()

    return listOfProb, zeroOccurenceProb, listOfCounts

In [ ]:
def convertTuple(tup): 
    str =  ''.join(tup) 
    return str

## Backoff

In [ ]:
def calc_backoff(tuple,length,ngramPrev,ngramPrevGoodTuring,ngram,ngramGoodTuring):

    pre_word=""
    if len(tuple) == 2:
        pre_word = (tuple[0],)
    elif len(tuple) == 3:
        pre_word = (tuple[0],tuple[1],)
    elif len(tuple) == 4:
        pre_word = (tuple[0],tuple[1],tuple[2],)    
    elif len(tuple) == 5:
        pre_word = (tuple[0],tuple[1],tuple[2],tuple[3],)  
        
    if tuple in ngram:
        return float(ngramGoodTuring[tuple])/ngramPrevGoodTuring[pre_word]
    else:
        total_prob = 0.0
        for key, value in ngram.items():
            if key == pre_word:              
                total_prob += float(ngramGoodTuring[key])/ngramPrev[key]
        alpha = 1-total_prob
        if alpha == 0: return -float("inf")
        else: return log(alpha*float(ngramPrevGoodTuring[pre_word])/length)

In [ ]:
def calculateNGramsProbWithBackoff(ngramPrevCount,nGramPrevGoodTuring,ngram,ngramGoodTuring): #gram1,unigramGoodTuring,gram2,bigramGoodTuring) 
    backOffProb = {}
    for key, value in ngram.items():  
        backOffProb[key] = calc_backoff(key,len(nGramPrevGoodTuring),ngramPrevCount,nGramPrevGoodTuring,ngram,ngramGoodTuring)
        #print(backOffProb[n[0]])
    return backOffProb

## 1 Gram

In [ ]:
gram1 = extract_ngrams(train_syllables,1)
print("1 Grams list:\n",gram1)
#print("1 Grams dictionary:\n",gram1Count)

1 Grams list:
 {('id',): 4, (' ',): 10080, ('url',): 1, ('tr',): 1, ('wi',): 2, ('ki',): 180, ('pe',): 40, ('di',): 416, ('a',): 366, ('org',): 1, ('cu',): 34, ('rid',): 1, ('tit',): 2, ('le',): 637, ('cen',): 230, ('giz',): 243, ('han',): 262, ('his',): 2, ('khan',): 1, ('çing',): 1, ('gis',): 1, ('ha',): 280, ('an',): 79, ('ya',): 306, ('da',): 484, ('do',): 35, ('ğum',): 5, ('dıy',): 3, ('la',): 738, ('te',): 303, ('mu',): 234, ('çin',): 307, ('mı',): 23, ('de',): 420, ('mir',): 6, ('ci',): 47, ('mo',): 102, ('ca',): 108, ('ten',): 36, ('niz',): 6, ('d',): 1, ('–',): 1, ('ö',): 70, ('ğus',): 3, ('tos',): 3, ('ko',): 57, ('tan',): 89, ('hü',): 23, ('küm',): 20, ('dar',): 38, ('ve',): 335, ('im',): 32, ('pa',): 38, ('ra',): 411, ('tor',): 21, ('lu',): 109, ('ğu',): 86, ('nun',): 83, ('ku',): 96, ('ru',): 64, ('su',): 80, ('dur',): 11, ('yüz',): 14, ('yı',): 94, ('lın',): 26, ('ba',): 161, ('şın',): 20, ('or',): 68, ('ta',): 252, ('as',): 39, ('tüm',): 10, ('gö',): 80, ('çe',): 43, ('b

In [ ]:
unigramGoodTuring, zeroOccurenceUnigramProb, unigramGoodTuringCstar = GoodTuring(gram1, len(gram1),'goodTuringDiscounting1.txt')
print(unigramGoodTuring)
print(zeroOccurenceUnigramProb)
print(unigramGoodTuringCstar)

{('id',): 0.0037735849056603774, (' ',): 0.13117699910152741, ('url',): 0.0011767309007523032, ('tr',): 0.0011767309007523032, ('wi',): 0.0025495119834742478, ('ki',): 0.09793351302785265, ('pe',): 0.012279125486672655, ('di',): 0.12578616352201258, ('a',): 0.12309074573225516, ('org',): 0.0011767309007523032, ('cu',): 0.018867924528301886, ('rid',): 0.0011767309007523032, ('tit',): 0.0025495119834742478, ('le',): 0.1293800539083558, ('cen',): 0.215633423180593, ('giz',): 0.1105121293800539, ('han',): 0.11230907457322552, ('his',): 0.0025495119834742478, ('khan',): 0.0011767309007523032, ('çing',): 0.0011767309007523032, ('gis',): 0.0011767309007523032, ('ha',): 0.11410601976639713, ('an',): 0.1293800539083558, ('ya',): 0.11859838274932614, ('da',): 0.1275831087151842, ('do',): 0.0862533692722372, ('ğum',): 0.005519188807598511, ('dıy',): 0.003788739865120861, ('la',): 0.1302785265049416, ('te',): 0.11680143755615453, ('mu',): 0.05435759209344115, ('çin',): 0.11949685534591195, ('mı',)

In [ ]:
ordered_dict = OrderedDict(sorted(unigramGoodTuring.items(), key = itemgetter(1), reverse = True))
out = list(ordered_dict.items())[0: 50]
#print(out)
randomlyChosedList=[]
for x in range(5):
  randomlyChosedList.append(random.choice(out))
randomlyChosedDict=dict(randomlyChosedList)
#print(randomlyChosedDict)
generated_sentence=""
for key, value in randomlyChosedDict.items():
  generated_sentence += convertTuple(key)
print("Genereted Sentence From 1 Gram:\n",generated_sentence)

Genereted Sentence From 1 Gram:
 tihangizcio


## 2 Gram

In [ ]:
gram2 =extract_ngrams(train_syllables,2)
print("2 Grams list:\n",gram2)

2 Grams list:
 {('id', ' '): 1, (' ', 'url'): 1, ('url', 'tr'): 1, ('tr', 'wi'): 1, ('wi', 'ki'): 2, ('ki', 'pe'): 1, ('pe', 'di'): 1, ('di', 'a'): 4, ('a', 'org'): 1, ('org', 'wi'): 1, ('ki', 'cu'): 1, ('cu', 'rid'): 1, ('rid', ' '): 1, (' ', 'tit'): 2, ('tit', 'le'): 1, ('le', 'cen'): 1, ('cen', 'giz'): 230, ('giz', ' '): 227, (' ', 'han'): 240, ('han', ' '): 149, (' ', 'cen'): 229, (' ', 'his'): 2, ('his', ' '): 1, (' ', 'khan'): 1, ('khan', ' '): 1, (' ', 'çing'): 1, ('çing', 'gis'): 1, ('gis', ' '): 1, (' ', 'ha'): 194, ('ha', 'an'): 1, ('an', ' '): 6, (' ', 'ya'): 162, ('ya', ' '): 82, (' ', 'da'): 126, ('da', ' '): 322, (' ', 'do'): 35, ('do', 'ğum'): 5, ('ğum', ' '): 5, (' ', 'a'): 293, ('a', 'dıy'): 2, ('dıy', 'la'): 3, ('la', ' '): 52, (' ', 'te'): 164, ('te', 'mu'): 123, ('mu', 'çin'): 121, ('çin', ' '): 185, (' ', 'an'): 73, ('an', 'la'): 11, ('la', 'mı'): 5, ('mı', ' '): 10, (' ', 'de'): 125, ('de', 'mir'): 3, ('mir', 'ci'): 2, ('ci', ' '): 17, (' ', 'mo'): 98, ('mo', 'ca'

In [ ]:
bigramGoodTuring, zeroOccurenceBigramProb, bigramGoodTuringCstar = GoodTuring(gram2, len(gram2),'goodTuringDiscounting2.txt')
print(bigramGoodTuring)
print(zeroOccurenceBigramProb)
print(bigramGoodTuringCstar)

{('id', ' '): 0.0001797623142733497, (' ', 'url'): 0.0001797623142733497, ('url', 'tr'): 0.0001797623142733497, ('tr', 'wi'): 0.0001797623142733497, ('wi', 'ki'): 0.00034621803443521886, ('ki', 'pe'): 0.0001797623142733497, ('pe', 'di'): 0.0001797623142733497, ('di', 'a'): 0.000753253233114171, ('a', 'org'): 0.0001797623142733497, ('org', 'wi'): 0.0001797623142733497, ('ki', 'cu'): 0.0001797623142733497, ('cu', 'rid'): 0.0001797623142733497, ('rid', ' '): 0.0001797623142733497, (' ', 'tit'): 0.00034621803443521886, ('tit', 'le'): 0.0001797623142733497, ('le', 'cen'): 0.0001797623142733497, ('cen', 'giz'): 0.020990764063811923, ('giz', ' '): 0.02065491183879093, (' ', 'han'): 0.02115869017632242, ('han', ' '): 0.01712846347607053, (' ', 'cen'): 0.020822837951301427, (' ', 'his'): 0.00034621803443521886, ('his', ' '): 0.0001797623142733497, (' ', 'khan'): 0.0001797623142733497, ('khan', ' '): 0.0001797623142733497, (' ', 'çing'): 0.0001797623142733497, ('çing', 'gis'): 0.0001797623142733

In [ ]:
bigramBackOffProb = calculateNGramsProbWithBackoff(gram1,unigramGoodTuring,gram2,bigramGoodTuring)  
sortedBigram = sorted(bigramBackOffProb.items(), key=lambda kv: kv[1],reverse=True)
sortedBigram

[(('ver', 'di'), 0.7401309823677581),
 (('ı', ' '), 0.7274014568724897),
 (('en', ' '), 0.702750629722922),
 (('din', ' '), 0.627455919395466),
 (('sin', 'de'), 0.6116784978245935),
 (('kar', 'şı'), 0.5574453751977037),
 (('lık', ' '), 0.551865341411941),
 (('kan', ' '), 0.5338027631478512),
 (('ten', ' '), 0.5152426986180135),
 (('sın', 'da'), 0.496362059710121),
 (('cak', ' '), 0.4941081096731232),
 (('tay', ' '), 0.4806045340050377),
 (('söy', 'le'), 0.46937829173345547),
 (('ü', 'ze'), 0.4438916876574307),
 (('yun', 'ca'), 0.4325643080680864),
 (('bü', 'yük'), 0.43201882974769795),
 (('fın', 'dan'), 0.39160369437447523),
 (('cek', ' '), 0.387153652392947),
 (('el', 'çi'), 0.387153652392947),
 (('uy', 'gur'), 0.3784760705289672),
 (('dir', ' '), 0.3735186152239357),
 (('bir', ' '), 0.3680081229375354),
 (('rek', ' '), 0.36421882064199446),
 (('ni', ' '), 0.3518150837161061),
 (('mo', ' '), 0.3477242106496397),
 (('mış', 'tı'), 0.3477242106496397),
 (('rak', ' '), 0.3426532325776658)

In [ ]:
out = sortedBigram[0: 50]
#print(out)
randomlyChosedList=[]
for x in range(5):
  randomlyChosedList.append(random.choice(out))
randomlyChosedDict=dict(randomlyChosedList)
#print(randomlyChosedDict)

generated_sentence=""
for key, value in randomlyChosedDict.items():
  generated_sentence += convertTuple(key)
print("Randomly Genereted Sentence From 2 Gram:\n",generated_sentence)

Randomly Genereted Sentence From 2 Gram:
 dir çirdirul nemlipekin


## 3 Gram

In [ ]:
gram3 =extract_ngrams(train_syllables,3)
#print("3 Grams list:\n",gram3)

In [ ]:
treeGoodTuring, zeroOccurenceTreegramProb, treegramGoodTuringCstar = GoodTuring(gram3, len(gram3),'goodTuringDiscounting3.txt')
print(treeGoodTuring)
print(zeroOccurenceTreegramProb)
print(treegramGoodTuringCstar)

{('id', ' ', 'url'): 4.4966716802785553e-05, (' ', 'url', 'tr'): 4.4966716802785553e-05, ('url', 'tr', 'wi'): 4.4966716802785553e-05, ('tr', 'wi', 'ki'): 4.4966716802785553e-05, ('wi', 'ki', 'pe'): 4.4966716802785553e-05, ('ki', 'pe', 'di'): 4.4966716802785553e-05, ('pe', 'di', 'a'): 4.4966716802785553e-05, ('di', 'a', 'org'): 4.4966716802785553e-05, ('a', 'org', 'wi'): 4.4966716802785553e-05, ('org', 'wi', 'ki'): 4.4966716802785553e-05, ('wi', 'ki', 'cu'): 4.4966716802785553e-05, ('ki', 'cu', 'rid'): 4.4966716802785553e-05, ('cu', 'rid', ' '): 4.4966716802785553e-05, ('rid', ' ', 'tit'): 4.4966716802785553e-05, (' ', 'tit', 'le'): 4.4966716802785553e-05, ('tit', 'le', 'cen'): 4.4966716802785553e-05, ('le', 'cen', 'giz'): 4.4966716802785553e-05, ('cen', 'giz', ' '): 0.004535581959882459, ('giz', ' ', 'han'): 0.004471700523827776, (' ', 'han', ' '): 0.004407819087773093, ('han', ' ', 'cen'): 9.80591956678591e-05, (' ', 'cen', 'giz'): 0.0045994633959371405, ('han', ' ', 'his'): 4.4966716

In [ ]:
treegramBackOffProb = calculateNGramsProbWithBackoff(gram2,bigramBackOffProb,gram3,treeGoodTuring)  
sortedtreegram = sorted(treegramBackOffProb.items(), key=lambda kv: kv[1],reverse=True)
sortedtreegram

[(('i', 'di', ' '), 0.305476162926245),
 (('o', 'nun', ' '), 0.23459091295390902),
 ((' ', 'oğ', 'lu'), 0.22658005894552827),
 ((' ', 'en', ' '), 0.21831932762980588),
 ((' ', 'çin', ' '), 0.17935633375467286),
 (('et', 'ti', ' '), 0.175583181919325),
 (('di', 'ği', ' '), 0.13215938424035215),
 (('rın', 'da', ' '), 0.13206823983742777),
 (('son', 'ra', ' '), 0.12116228906783669),
 ((' ', 'sal', 'dı'), 0.11455135571183339),
 (('la', 'ra', ' '), 0.10656969954675455),
 ((' ', 'ken', 'di'), 0.10414238734142293),
 (('o', 'na', ' '), 0.10271577929565001),
 (('o', 'lan', ' '), 0.09975688853404693),
 (('bu', 'ha', 'ra'), 0.09137746244777112),
 (('ği', 'ni', ' '), 0.08715683529643914),
 (('o', 'lup', ' '), 0.08197016200558961),
 (('ği', 'nin', ' '), 0.08195637742129296),
 (('ği', 'ne', ' '), 0.08164996606025526),
 (('re', 'rek', ' '), 0.08059084048050515),
 (('du', 'ğu', ' '), 0.07929563054421129),
 ((' ', 'uy', 'gur'), 0.07800401841329607),
 (('baş', 'ka', ' '), 0.0778694897381006),
 (('de', '

In [ ]:
out = sortedtreegram[0: 50]
#print(out)
randomlyChosedList=[]
for x in range(5):
  randomlyChosedList.append(random.choice(out))
randomlyChosedDict=dict(randomlyChosedList)
#print(randomlyChosedDict)

generated_sentence=""
for key, value in randomlyChosedDict.items():
  generated_sentence += convertTuple(key)
print("Randomly Genereted Sentence From 3 Gram:\n",generated_sentence)

Randomly Genereted Sentence From 3 Gram:
 onun  en başka diye buhara


## 4 Gram

In [ ]:
gram4 = extract_ngrams(train_syllables,4)
#print("4 Grams list:\n",gram4)

In [ ]:
fourGoodTuring, zeroOccurenceFourgramProb, fourgramGoodTuringCstar = GoodTuring(gram4, len(gram4),'goodTuringDiscounting4.txt')
#print(fourGoodTuring, zeroOccurenceFourgramProb, fourgramGoodTuringCstar)

In [ ]:
fourgramBackOffProb = calculateNGramsProbWithBackoff(gram3,treegramBackOffProb,gram4,fourGoodTuring)
sortedFourgram = sorted(fourgramBackOffProb.items(), key=lambda kv: kv[1],reverse=True)
sortedFourgram

[((' ', 'han', 'a', ' '), 0.672305110735396),
 (('din', ' ', 'mu', 'ham'), 0.402351096273006),
 (('ver', 'di', 'ler', ' '), 0.38574158197398123),
 (('ı', ' ', 'yok', ' '), 0.37910720587123464),
 (('ı', ' ', 'e', 'le'), 0.37910720587123464),
 (('en', ' ', 'ya', 'kın'), 0.37314502871724226),
 (('en', ' ', 'kü', 'çük'), 0.37314502871724226),
 (('en', ' ', 'ö', 'nem'), 0.37314502871724226),
 (('kar', 'şı', 'sın', 'da'), 0.3696344882131225),
 ((' ', 'gö', 're', ' '), 0.3675377390068311),
 (('en', ' ', 'faz', 'la'), 0.36625968389448726),
 (('en', ' ', 'zen', 'gin'), 0.36625968389448726),
 (('en', ' ', 'cen', 'giz'), 0.36625968389448726),
 (('lık', ' ', 'ta', 'ri'), 0.3659344432860643),
 (('sin', 'de', 'ki', ' '), 0.36477186588358257),
 (('kar', 'şı', 'laş', 'ma'), 0.33643119652325093),
 (('din', ' ', 'em', 'ri'), 0.3270175749057922),
 (('din', ' ', 'bu', ' '), 0.3270175749057922),
 (('din', ' ', 'ra', 'zi'), 0.3270175749057922),
 ((' ', 'i', 'se', ' '), 0.3193016030941443),
 (('kan', ' ', 'h

In [ ]:
out = sortedFourgram[0: 50]
#print(out)
randomlyChosedList=[]
for x in range(5):
  randomlyChosedList.append(random.choice(out))
randomlyChosedDict=dict(randomlyChosedList)
#print(randomlyChosedDict)

generated_sentence=""
for key, value in randomlyChosedDict.items():
  generated_sentence += convertTuple(key)
print("Randomly Genereted Sentence From 4 Gram:\n",generated_sentence)

Randomly Genereted Sentence From 4 Gram:
 söyledi ı baş ı eleı kutlaı kera


## 5 Gram

In [ ]:
gram5 = extract_ngrams(train_syllables,5)
#print("5 Grams list:\n",gram5)

In [ ]:
fiveGoodTuring, zeroOccurenceFivegramProb, fivegramGoodTuringCstar = GoodTuring(gram5, len(gram5),'goodTuringDiscounting5.txt')
#print(fiveGoodTuring, zeroOccurenceFivegramProb, fivegramGoodTuringCstar)

In [ ]:
fivegramBackOffProb = calculateNGramsProbWithBackoff(gram4,fourgramBackOffProb,gram5,fiveGoodTuring)
sortedFivegram = sorted(fivegramBackOffProb.items(), key=lambda kv: kv[1],reverse=True)
sortedFivegram

[(('i', 'di', ' ', 'ke', 'ra'), 0.2128013829924692),
 (('i', 'di', ' ', 'bu', ' '), 0.2128013829924692),
 (('i', 'di', ' ', 'su', 'bu'), 0.2128013829924692),
 (('i', 'di', ' ', 'ot', 'rar'), 0.2128013829924692),
 (('i', 'di', ' ', 'te', 'mu'), 0.21276505473415613),
 (('i', 'di', ' ', 'cen', 'giz'), 0.21276505473415613),
 ((' ', 'kar', 'de', 'şi', ' '), 0.1968480849479873),
 (('o', 'nun', ' ', 'ta', 'ra'), 0.16342116594580547),
 ((' ', 'oğ', 'lu', 'na', ' '), 0.15784062965909765),
 ((' ', 'en', ' ', 'faz', 'la'), 0.15208602336944305),
 ((' ', 'en', ' ', 'zen', 'gin'), 0.15208602336944305),
 ((' ', 'en', ' ', 'ya', 'kın'), 0.15206006009671857),
 ((' ', 'en', ' ', 'kü', 'çük'), 0.15206006009671857),
 ((' ', 'en', ' ', 'ö', 'nem'), 0.15206006009671857),
 (('i', 'di', ' ', 'tuğ', 'rul'), 0.1513575081539948),
 (('i', 'di', ' ', 'so', 'nun'), 0.1513575081539948),
 (('i', 'di', ' ', 'u', 'zak'), 0.1513575081539948),
 (('i', 'di', ' ', 'ko', ' '), 0.1513575081539948),
 (('i', 'di', ' ', 'ku', '

In [ ]:
out = sortedFivegram[0: 50]
#print(out)
randomlyChosedList=[]
for x in range(5):
  randomlyChosedList.append(random.choice(out))
randomlyChosedDict=dict(randomlyChosedList)
#print(randomlyChosedDict)

generated_sentence=""
for key, value in randomlyChosedDict.items():
  generated_sentence += convertTuple(key)
print("Randomly Genereted Sentence From 5 Gram:\n",generated_sentence)

Randomly Genereted Sentence From 5 Gram:
 onun kardeonun hale en fazlaidi ve 


## Perplexity

In [ ]:
encoder = Encoder(lang="tr", limitby="vocabulary", limit=3000)  # split into syllables (also cleans the text)

tokens= joined_test_articles[0].split()

test_syllables = ""
for words in tokens:
  if encoder.tokenize(words) != "":
    test_syllables += encoder.tokenize(words).replace(" ", ",") + ", ,"
test_syllables = test_syllables[:-3]

In [ ]:
def pairwise(n,iterable):
    a = iter(iterable)
    return  zip(*[iter(iterable)]*n)

listTestSentences = test_syllables[0:20].split(',') 

def calculatePerplexity(n,ngramProbality,testCorpus):
  index = 1
  perplexity = 0
  size = 500
  listTestSentences =testCorpus
  count = 0
  for hece  in pairwise(n,testCorpus[0:size].split(',')):
    if hece in ngramProbality.keys():
      perplexity+=log(float(ngramProbality[hece]),2)    
    count+=1
  print("Perplexity for "+ str(n) +" gram: " + str(pow(-1/(perplexity/count),2)))

In [ ]:
calculatePerplexity(1,unigramGoodTuring,test_syllables)

Perplexity for 1 gram: 0.06967958875800374


In [ ]:
calculatePerplexity(2,bigramGoodTuring,test_syllables)

Perplexity for 2 gram: 0.020338401407144307


In [ ]:
calculatePerplexity(3,treeGoodTuring,test_syllables)

Perplexity for 3 gram: 0.016961000448054816


In [ ]:
calculatePerplexity(4,fourGoodTuring,test_syllables)

Perplexity for 4 gram: 0.028303389664423636


In [ ]:
calculatePerplexity(5,fiveGoodTuring,test_syllables)

Perplexity for 5 gram: 0.031938630101745115
